In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn

In [106]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(100):
                    results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetTemizle(DataSet)
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
                  
    def Ceviri(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
                DataSet['Tweet_Ceviri'][yazi]=DataSet['Tweet_Ceviri'][yazi].lower()
            print ("Tweetler Çevirildi!")                        
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            DataSet["Tür"] = DataSet["Tweet_Ceviri"].apply(getTweets.getPolarity)
            DataSet["Yoğunluk"] = DataSet["Tweet_Ceviri"].apply(getTweets.getSubjectivity)
            
    def TweetTemizle(Dataset):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
            for i in range(len(Dataset["Tweet_Degistirilmis"])):        
                Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])    
                Dataset["Tweet_Degistirilmis"][i]=Dataset["Tweet_Degistirilmis"][i].lower()                
            Dataset["Tweet_Degistirilmis"]=Dataset["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")            
            return Dataset
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.Ceviri(Dataset)
    
    def KelimeAyristirma(DataSet):
        try:
            data_test = DataSet.copy()
            stop = stopwords.words('english')
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])
        except Exception as err:
            print('Error: {}'.format(err))
    
    def turkceAnaliz(self,Dataset):
        return 
    
    def getSubjectivity(Dataset):
        try:
            return TextBlob(Dataset).sentiment.subjectivity
        except Exception as err:
            print('Error: {}'.format(err))
    
    def getPolarity(Dataset):
        try:
            return TextBlob(Dataset).sentiment.polarity 
        except Exception as err:
            print('Error: {}'.format(err))

In [138]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.TextBlob'>
Error

In [139]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Tür,Yoğunluk
0,1611156450008760320,@Fmnkr Eskiden cebinde 10 TL olduğunda kendini lord hissettiğin o günlerden çok uzaktayiz dostum\n10 TL ile akbiline… https://t.co/9OfzhjdkBY,eskiden cebinde tl olduğunda kendini lord hissettiğin o günlerden çok uzaktayiz dostum tl ile akbiline,"in the past, when you were in your pocket, we are far from those days when you feel lord, my friend tl.",2023-01-06 00:23:26+00:00,1465323758126874626,Ali Kurtdere,tr,NaN,NaN
1,1611154431541915649,@Osman64145807 @BRCNYLDRM78 Bak hele şuna. Cola alamamış. Yahu fırın ve buzdolabı bulamıyorduk biz. Sahi sen hangi… https://t.co/F5jeNP7Nn8,bak hele şuna cola alamamış yahu fırın ve buzdolabı bulamıyorduk biz sahi sen hangi,"look, we couldn't find a bakery and a refrigerator.",2023-01-06 00:15:24+00:00,254962755,izmirli,tr,NaN,NaN
2,1611153421910036480,"82 milyon nüfusun 62 milyonu coca-cola, iphone, starbucks, mcdonald's, marlboro peşindeyken, 60 küsür hilton otel v… https://t.co/N7hpAhrxG7",milyon nüfusun milyonu coca cola iphone starbucks mcdonald s marlboro peşindeyken küsür hilton otel v,million of million population coca cola iphone starbucks mcdonald s marlboro is in pursuit of hilton hotel v,2023-01-06 00:11:24+00:00,111864335,Ayhan Öztürk,tr,NaN,NaN
3,1611139032423043078,"RT @hakanchelik: Coca Cola, Vodafone gibi dünya devlerinden sonra dünyanın en başarılı havacılık ve mühendislik şirketlerinden Birleşik Kra…",rt coca cola vodafone gibi dünya devlerinden sonra dünyanın en başarılı havacılık ve mühendislik şirketlerinden birleşik kra,"after world giants such as rt coca cola vodafone, united kra from the world's most successful aviation and engineering companies",2023-01-05 23:14:13+00:00,1430818003462000640,İslam oğlu,tr,NaN,NaN
4,1611131751866195970,@semihayas88 Warren buffet Coca Cola hissesini 20 yıl tutmuş. Uzun vadeci olmamda bunun çok etkisi var. Dile kolay… https://t.co/zWBJC8wwmq,warren buffet coca cola hissesini yıl tutmuş uzun vadeci olmamda bunun çok etkisi var dile kolay,"warren buffet coca cola is a long valley who has been a year of years, it has a lot of effect.",2023-01-05 22:45:17+00:00,823635847652188160,selcuk gonultas,tr,NaN,NaN


In [140]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Tür,Yoğunluk
95,1610971298154291202,Atay Sözer: Coca Cola Türkiye https://t.co/J0Jo9rtnTl @istanbulgercegi @homurmizah https://t.co/v1js7VeRGx,atay sözer coca cola türkiye,atay sözer coca cola turkey,2023-01-05 12:07:42+00:00,302012522,İmambakır Üküş,tr,NaN,NaN
96,1610971101332488192,Atay Sözer: Coca Cola Türkiye https://t.co/8Y32RqjONV @istanbulgercegi @homurmizah https://t.co/tWo7ubutWO,atay sözer coca cola türkiye,atay sözer coca cola turkey,2023-01-05 12:06:55+00:00,289218059,İstanbul Gerçeği,tr,NaN,NaN
97,1610970914069577729,Atay Sözer: Coca Cola Türkiye https://t.co/OzTxQOuXMQ @istanbulgercegi @homurmizah https://t.co/1KObzC3Rpv,atay sözer coca cola türkiye,atay sözer coca cola turkey,2023-01-05 12:06:10+00:00,284012988,Dilfiraz Değerli,tr,NaN,NaN
98,1610968605184729090,"doğru olduğunu sanmam da, doğruysa da Dünyanın en mantıksız kararı ... yere atılan kola şişelerinin temizleme harca… https://t.co/pwoAHjqmDW",doğru olduğunu sanmam da doğruysa da dünyanın en mantıksız kararı yere atılan kola şişelerinin temizleme harca,"although it is true that i think it is true, the most unreasonable decision in the world is to clean the cola bottles",2023-01-05 11:57:00+00:00,64828520,Oguz Sahin ★★★★,tr,NaN,NaN
99,1610966745640235008,"@msmsnxnxms @z3hir2 @absurtolaylar Le cola, simbat çekirdek, diriliş Ertuğrul üçlüsünü seven bir kitlenin hayatının… https://t.co/OuBm3N0JFs",le cola simbat çekirdek diriliş ertuğrul üçlüsünü seven bir kitlenin hayatının,le cola simbat kernel resurrection ertuğrul's life of a mass that loves the trio,2023-01-05 11:49:37+00:00,1925932400,şahin Bozkurt,tr,NaN,NaN


In [141]:
"""savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(tw.getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(tw.getPolarity)
savedTweets['Duygu Turkce'] = savedTweets['Tweet_Ceviri'].apply(tw.turkceAnaliz)
savedTweets.to_csv('duygu.csv')
savedTweets.head()"""

"savedTweets = Veri\nsavedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(tw.getSubjectivity)\nsavedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(tw.getPolarity)\nsavedTweets['Duygu Turkce'] = savedTweets['Tweet_Ceviri'].apply(tw.turkceAnaliz)\nsavedTweets.to_csv('duygu.csv')\nsavedTweets.head()"